In [1]:
import sys
import os
import re
from query_rdfs import *
import itertools

sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
current_dir = os.path.dirname("__file__")
current_dir = current_dir if current_dir is not '' else '.'
# directory to scan for any txt files
data_dir_path = current_dir + '/rule'


known = {"name": "Che Viet Hai", "height": 170, "weight": 55, "gender": "nam", 
         "age": 17, "PhysicalActiveLevel": "Hoạt động vừa", "target": "giữ cân",
         "ListIllness": "Bệnh béo phì"}
data_rule = []
function = []

In [2]:
with open(data_dir_path + "/rule.txt", "r", encoding="utf8") as f:
    for line in f:
        data_rule.append(line)
with open(data_dir_path + "/function.txt", "r", encoding="utf8") as f:
    for line in f:
        function.append(line)

In [3]:
print(function)

['\ufeffbmr,nu=447.593+(9.247*weight)+(3.098*height)-(4.33*age)\n', 'bmr,nam=88.362+(13.397 *weight)+(4.799*height)-(5.677*age)\n', 'bmi=weight/(height*height)\n', 'calo=BMRvalue*PhysicalActiveValue\n', '1.2:Quá ít hoạt động\n', '1.375:Ít hoạt động\n', '1.55:Hoạt động vừa\n', '1.725:Hoạt động nhiều\n', '1.9:Hoạt động cường độ cao']


# BMI function

In [4]:
def BMIvalueCalc(height, weight):
    height/=100
    return round(eval(function[2].split("=")[1]),2)

def BMIrangeCalc(bmi_value):
    return BMI_range_calc(bmi_value)

def BMIlevelCalc(bmi_range):
    return has_value(bmi_range)["value"]

# BMR function

In [5]:
def BMRvalueOfMaleCalc(height, weight, age):
    function[1] = function[1].split("=")[1]
    return round(eval(function[1]), 2)

def BMRvalueOfFemaleCalc(height, weight, age):
    function[1] = function[1].split("=")[1]
    return round(eval(function[1]), 2)

# print (BMIvalueCalc(height,weight)) 

# PA function

In [6]:
def PAvalueCalc(PhysicalActiveLevel):
    for count in range(4,9):
        result = function[count].split(":")
        if result[1].endswith("\n"):
            result[1] = result[1][:-1]
        if  result[1] == PhysicalActiveLevel:
            return float(result[0])

# CaloCalc function

In [7]:
def CaloCalc(BMRvalue, PhysicalActiveValue, target):
    result = round(eval(function[3].split("=")[1]),2)
    if target == "tăng cân":
        result +=500
    elif target == "giảm cân":
        result -=500
    return result

def CaloToKeepWeightCalc(calo_per_day):
    # breakfast: 26%, #Lunch: 40%, # Dinner: 34%
    return [round(calo_per_day*26/100, 2), 
            round(calo_per_day*40/100, 2), round(calo_per_day*34/100, 2)]    

# ListFood function

In [8]:
def FindAvoidFood(list_illness):
    return avoided_group_food(list_illness) # return array

def FindNeedFood(list_illness):
    return needed_group_food(list_illness)

def FindLimitFood(list_illness):
    return limit_food(list_illness)

def FilterFood(list_avoid_food, list_need_food, 
               list_limit_food):
    result = {}
    list_all_food = []
    print(all_food())
    tamp_list = [food.split("|") for food in all_food()]
    set_avoid_food = set(list_avoid_food)
    for food in tamp_list:
        if food[0] not in set_avoid_food:
            list_all_food.append(food)
    print(list_all_food)
    for food_1 in list_all_food:
        if food_1[0] in list_need_food:
            food_1.append(1)
            result[food_1[0]] = food_1[1:]
        elif food_1[0] in list_limit_food:
            food_1.append(-1)
            result[food_1[0]] = food_1[1:]
        else:
            food_1.append(0)
            result[food_1[0]] = food_1[1:]
    return result

# CreateMenu Function

In [9]:
def extract_rule_left(rule_left):
    rule_left = re.findall(r'\[(.*?)\]',rule_left)
    flag = True
    for content in rule_left:
        content = content.split(",")
        if content[0] == "1":
            if content[1] not in known:
                flag = False
                break
        elif content[0] == "2":
            try:
                if known[content[1]] != content[2]:
                    flag = False
                    break
            except:
                flag = False
                break
    return flag
                
def extract_rule_right(rule_right):
    rule_right = rule_right.split(",", 1)
    flag = True
    rule_right[0] = rule_right[0][1:]
    rule_right[1] = rule_right[1][:-2]
    if rule_right[0] in known:
        flag = False
    else:
        funct = rule_right[1].split(':')[1]
        known[rule_right[0]] = eval(funct)
    return flag
    
def extract_rule(rule):
    flag = True
    rule_left, rule_right = rule.split("|")
    flag = extract_rule_left(rule_left)
    if flag is False:
        return flag
    else: 
        flag = extract_rule_right(rule_right)
    return flag

In [10]:
count = 0
while count <= 13:
    if extract_rule(data_rule[count]) == True:
        print("True")
        count = 0
    else: count+=1        

True
True
True
True
True
True
True
True
True
True
['Süt|Dairy_Products|32|', 'Atiso|Vegetables|47|', 'Bí_đỏ|Vegetables|559|', 'Bơ|Fruits|160|', 'Bạc_hà|Vegetables|70|', 'Bắp|Fruits|86|', 'Bắp_cải|Vegetables|25|', 'Chanh|Fruits|29|', 'Cà_chua|Fruits|18|', 'Cà_rốt|Vegetables|41|', 'Cà_tím|Vegetables|24|', 'Cần_tây|Vegetables|16|', 'Củ_hành|Vegetables|40|', 'Củ_sắn|Vegetables|38|', 'Dâu_tây|Fruits|32|', 'Dưa_hấu|Fruits|30|', 'Dưa_leo|Vegetables|15|', 'Dừa|Fruits|354|', "Gừng|Vegetables|80|['Thịt_chó', 'Thịt_cá', 'Thịt_thỏ']", 'Hẹ|Vegetables|30|', 'Khóm|Fruits|50|', 'Măng|Vegetables|27|', 'Nho|Fruits|299|', 'Rau_muốn|Vegetables|19|', 'Sầu_riêng|Fruits|147|', 'Thơm|Fruits|50|', 'Thịt_bò|Meat|254|', 'Thịt_cừu|Meat|282|', "Thịt_gà|Meat|143|['Cua_biển', 'Tôm']", 'Thịt_gà_tây|Meat|149|', 'Thịt_heo|Meat|243|', 'Thịt_lợn_rừng|Meat|160|', 'Thịt_vịt|Meat|132|', 'Táo|Fruits|52|', 'Xoài|Fruits|70|', 'Xà_lách|Vegetables|41|', 'Đu_Đủ|Fruits|43|', 'Đậu_nành|Vegetables|109|', 'Ổi|Fruits|68|']
[['Süt', 'D

In [40]:
print(extract_rule(data_rule[13]))

False


In [11]:
print(known)

{'name': 'Che Viet Hai', 'height': 170, 'weight': 55, 'gender': 'nam', 'age': 17, 'PhysicalActiveLevel': 'Hoạt động vừa', 'target': 'giữ cân', 'ListIllness': 'Bệnh béo phì', 'BMIvalue': 19.03, 'BMIrange': '18.50-24.99', 'BMIlevel': 'Bình_thường', 'BMRvalue': 1544.52, 'PhysicalActiveValue': 1.55, 'CaloPerDay': 2394.01, 'CaloForEachMeal': [622.44, 957.6, 813.96], 'ListAvoidFood': ['Butter', 'Peanut', 'Khoai_tây'], 'ListNeedFood': ['Bắp', 'Cà_rốt', 'Mực', 'Nho', 'Dưa_hấu', 'Táo'], 'ListLimitFood': ['Butter'], 'ListFood': {'Süt': ['Dairy_Products', '32', '', 0], 'Atiso': ['Vegetables', '47', '', 0], 'Bí_đỏ': ['Vegetables', '559', '', 0], 'Bơ': ['Fruits', '160', '', 0], 'Bạc_hà': ['Vegetables', '70', '', 0], 'Bắp': ['Fruits', '86', '', 1], 'Bắp_cải': ['Vegetables', '25', '', 0], 'Chanh': ['Fruits', '29', '', 0], 'Cà_chua': ['Fruits', '18', '', 0], 'Cà_rốt': ['Vegetables', '41', '', 1], 'Cà_tím': ['Vegetables', '24', '', 0], 'Cần_tây': ['Vegetables', '16', '', 0], 'Củ_hành': ['Vegetables', '

In [97]:
def find_type(list_food, types):
    result = {}
    for food in list_food:
        if list_food[food][0] == types:
            result[food] = list_food[food]
    return result

def FindListBreakFast(list_food):
    structure_breakfast = list_structure_breakfast(list_food["only_food_name"])
    print(structure_breakfast)

def FindlistDinner(list_food):
    structure_dinner = list_structure_dinner(list_food["only_food_name"])
    print(structure_dinner)
    
def FindlistLunch(known):
    list_food = known["ListFood"]
    calor_each_meal = known["CaloForEachMeal"][1]
    
    result = {}
    structure_lunch = list_structure_lunch(list_food)
    for element in structure_lunch:
        for key in element:
            if key == "fix" and len(element["fix"][0]) ==1:
                food = element["fix"][0][0]
                calor_tamp = calor_each_meal * element["fix"][1]
                portion = round(calor_tamp / float(list_food[food][1]) * 100, 1)
                result[food] = [calor_tamp, portion]
                for food in element["fix"][0]:
                    list_food[food] 
            else:
                types = key
                print(types)
                list_food_types = find_type(list_food, types)
                print(list_food_types)
                sorted_by_value = dict(sorted(list_food_types.items(), key=lambda kv: kv[1][1]))
                print(sorted_by_value)
                for count, key in enumerate(sorted_by_value):
                    if count > 3: break
                    else:
                        result[key] = sorted_by_value[key]                         
    print(structure_lunch)
    print(result)

In [98]:
print (FindlistLunch(known))

Egg_Products
{'Süt': ['Dairy_Products', '32', '', 0], 'Atiso': ['Vegetables', '47', '', 0], 'Bí_đỏ': ['Vegetables', '559', '', 0], 'Bơ': ['Fruits', '160', '', 0], 'Bạc_hà': ['Vegetables', '70', '', 0], 'Bắp': ['Fruits', '86', '', 1], 'Bắp_cải': ['Vegetables', '25', '', 0], 'Chanh': ['Fruits', '29', '', 0], 'Cà_chua': ['Fruits', '18', '', 0], 'Cà_rốt': ['Vegetables', '41', '', 1], 'Cà_tím': ['Vegetables', '24', '', 0], 'Cần_tây': ['Vegetables', '16', '', 0], 'Củ_hành': ['Vegetables', '40', '', 0], 'Củ_sắn': ['Vegetables', '38', '', 0], 'Dâu_tây': ['Fruits', '32', '', 0], 'Dưa_hấu': ['Fruits', '30', '', 1], 'Dưa_leo': ['Vegetables', '15', '', 0], 'Dừa': ['Fruits', '354', '', 0], 'Gừng': ['Vegetables', '80', "['Thịt_chó', 'Thịt_cá', 'Thịt_thỏ']", 0], 'Hẹ': ['Vegetables', '30', '', 0], 'Khóm': ['Fruits', '50', '', 0], 'Măng': ['Vegetables', '27', '', 0], 'Nho': ['Fruits', '299', '', 1], 'Rau_muốn': ['Vegetables', '19', '', 0], 'Sầu_riêng': ['Fruits', '147', '', 0], 'Thơm': ['Fruits', '50',

In [21]:
print ([key for key in known])

['name', 'height', 'weight', 'gender', 'age', 'PhysicalActiveLevel', 'target', 'ListIllness', 'BMIvalue', 'BMIrange', 'BMIlevel', 'BMRvalue', 'PhysicalActiveValue', 'CaloPerDay', 'CaloForEachMeal', 'ListAvoidFood', 'ListNeedFood', 'ListLimitFood', 'ListFood']


In [9]:
calo / percent

[2, 3]


In [63]:
tamp = list(itertools.combinations(['a','b','c'],1))
for i in tamp:
    print(i)
print(list(itertools.combinations(['a','b','c'],1)))
[''.join(x) for x in itertools.combinations(['a','b','c'],2)]

('a',)
('b',)
('c',)
[('a',), ('b',), ('c',)]


['ab', 'ac', 'bc']

In [40]:
tamp = {'fix': [['Bơ'], 0.4, 1], 'Egg_Products': [0.3, 1], 'Fruit': [0.3, 1]}
print(tamp['fix'][0][0])

Bơ


In [84]:
x = {'1': [1,2], 3: [3,4], 4: [10,0]}
sorted_by_value = dict(sorted(x.items(), key=lambda kv: kv[1][1]))

for count, key in enumerate(sorted_by_value):
    if count > 3: break
    else:
        print(key)

4
1
3


In [77]:


print (find_type(known["ListFood"], "Fruits"))

{'Bơ': ['Fruits', '160', '', 0], 'Bắp': ['Fruits', '86', '', 1], 'Chanh': ['Fruits', '29', '', 0], 'Cà_chua': ['Fruits', '18', '', 0], 'Dâu_tây': ['Fruits', '32', '', 0], 'Dưa_hấu': ['Fruits', '30', '', 1], 'Dừa': ['Fruits', '354', '', 0], 'Khóm': ['Fruits', '50', '', 0], 'Nho': ['Fruits', '299', '', 1], 'Sầu_riêng': ['Fruits', '147', '', 0], 'Thơm': ['Fruits', '50', '', 0], 'Táo': ['Fruits', '52', '', 1], 'Xoài': ['Fruits', '70', '', 0], 'Đu_Đủ': ['Fruits', '43', '', 0], 'Ổi': ['Fruits', '68', '', 0]}
